In [ ]:
# create megadataset with narrowed down features 
'''
Customer Id
Article ID
Product Code
Product Type No
Index Group No
Perceived Color Value
Fashion News Frequency
Date
Price
Sales Channel ID
Postal Code
Season
Frequent shopper
Customer price point
Age group
'''
import pandas as pd

articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv', encoding='latin1',
                        index_col=["article_id"], 
                        usecols=["article_id", "product_code", "product_type_no", "index_group_no", "perceived_colour_value_id"])
'''
customers = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv', encoding='latin1',
                        index_col=["customer_id"], 
                        usecols=["customer_id", "fashion_news_frequency", "age"])

transactions = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv', encoding='latin1',
                          usecols=["customer_id", "article_id", "t_dat", "price", "sales_channel_id"])
'''
print(articles.isna().sum())

# 995 articles that have no transaction history
# using inner join to drop the 995 articles to have no NA's
#outer_merged = pd.merge(articles, transactions, how="inner", on=["article_id"])
#outer_merged.head()

#print(outer_merged.isna().sum())
#print(outer_merged.shape)

In [ ]:
# Libraries
import os
import gc
import wandb
import time
import random
import math
import glob
from scipy import spatial
from tqdm import tqdm
import warnings
import cv2
import pandas as pd
import numpy as np
from numpy import dot, sqrt
import seaborn as sns
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from IPython.display import display_html
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
plt.rcParams.update({'font.size': 16})


In [ ]:
# Environment check
warnings.filterwarnings("ignore")
os.environ["WANDB_SILENT"] = "true"
CONFIG = {'competition': 'HandM', '_wandb_kernel': 'aot'}

In [ ]:
!pip install turicreate --user

import turicreate as tc
import cudf
import cuml
import cupy

from cuml.model_selection import train_test_split

> MEGA DATASET

In [ ]:
transactions = outer_merged
transactions['customer_id'] =transactions['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')

In [ ]:
transactions['article_id'] = transactions['article_id'].astype('int32')
# BEFORE SUBMISSION TODO 
# train['article_id'] = '0' + train.article_id.astype('str')
transactions.head(5)

In [ ]:
TOP_CUSTOMERS = 300000
TOP_N = 200

In [ ]:
most_frequent_articles = transactions["article_id"].value_counts().reset_index()
most_frequent_articles.columns = ["article_id", "count"]
print("Total Unique IDs in Transactions:", len(most_frequent_articles))
print("Total Unique IDs that are selected:", TOP_N)

In [ ]:
most_frequent_articles = cupy.asarray(most_frequent_articles.head(TOP_N)["article_id"])

transactions = transactions[transactions["article_id"].isin(most_frequent_articles)].reset_index(drop=True)